In [1]:
!pip install mlflow

In [2]:
import mlflow

In [2]:
import numpy as np
from sklearn.linear_model import LogisticRegression

import mlflow
import mlflow.sklearn

if __name__ == "__main__":
    X = np.array([-2, -1, 0, 1, 2, 1]).reshape(-1, 1)
    y = np.array([0, 0, 1, 1, 1, 0])
    lr = LogisticRegression()
    lr.fit(X, y)
    score = lr.score(X, y)
    print("Score: %s" % score)
    mlflow.log_metric("score", score)
    mlflow.sklearn.log_model(lr, "model")
    print("Model saved in run %s" % mlflow.active_run().info.run_uuid)

Score: 0.6666666666666666
Model saved in run d813b96afead4d3998f27f5feafbfeb5


/opt/homebrew/anaconda3/lib/python3.8/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [12]:

import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file from the URL
    csv_url = (
        "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    )
    try:
        data = pd.read_csv(csv_url, sep=";")
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e
        )

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    alpha = 0.01
    l1_ratio = 0.5
    
#     models = [ElasticNet(), LinearRegression(), KNeighborsRegressor(), DecisionTreeRegressor(), RandomForestRegressor()]

#     with mlflow.start_run():
        
#         for alg in models:
            
        
#             alg.fit(train_x, train_y)

#             predicted_qualities = alg.predict(test_x)

#             (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

#             print("Model = ",alg)
#             print("  RMSE: %s" % rmse)
#             print("  MAE: %s" % mae)
#             print("  R2: %s" % r2)

#             mlflow.log_metric("rmse", rmse)
#             mlflow.log_metric("r2", r2)
#             mlflow.log_metric("mae", mae)

#             tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

#             # Model registry does not work with file store
#             if tracking_url_type_store != "file":

#                 # Register the model
#                 # There are other ways to use the Model Registry, which depends on the use case,
#                 # please refer to the doc for more information:
#                 # https://mlflow.org/docs/latest/model-registry.html#api-workflow
#                 mlflow.sklearn.log_model(alg, "model", registered_model_name=alg)
#             else:
#                 mlflow.sklearn.log_model(alg, "model")

In [5]:
mlflow.set_experiment("wine experiment").experiment_id

'1'

In [9]:
mlflow.end_run()
from sklearn.neighbors import KNeighborsRegressor


In [16]:
experiment_id = mlflow.set_experiment("wine experiment").experiment_id
k = [1,3,5,7,9,11,13,15,16]
for n_neigh in k:
        with mlflow.start_run(experiment_id=experiment_id):
            
            model = KNeighborsRegressor(n_neighbors=n_neigh)
            model.fit(train_x, train_y)
            y_pred = model.predict(test_x)
            mse = mean_squared_error(test_y, y_pred)
            r2 = r2_score(test_y, y_pred)       
            mlflow.log_param("n_neighbors",n_neigh)

            mlflow.log_metric("mse", mse)
            mlflow.log_metric("r2", r2)
            
            mlflow.sklearn.log_model(model,"model")

In [20]:
models = [ElasticNet(), LinearRegression(), KNeighborsRegressor(), DecisionTreeRegressor(), RandomForestRegressor()]

for alg in models:
        with mlflow.start_run(experiment_id=experiment_id):

                    alg.fit(train_x, train_y)

                    predicted_qualities = alg.predict(test_x)

                    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

                    print("Model = ",alg)
                    print("  RMSE: %s" % rmse)
                    print("  MAE: %s" % mae)
                    print("  R2: %s" % r2)
                    mlflow.log_param("model",alg)
                    mlflow.log_metric("rmse", rmse)
                    mlflow.log_metric("r2", r2)
                    mlflow.log_metric("mae", mae)

                    mlflow.sklearn.log_model(alg, "model")


Model =  ElasticNet()
  RMSE: 0.8328241218819784
  MAE: 0.6681571546815984
  R2: 0.017256182227355366
Model =  LinearRegression()
  RMSE: 0.6603811224461613
  MAE: 0.5073644199888914
  R2: 0.3820931219110172
Model =  KNeighborsRegressor()
  RMSE: 0.7721398836998384
  MAE: 0.581
  R2: 0.15525486167688018
Model =  DecisionTreeRegressor()
  RMSE: 0.8108637370113427
  MAE: 0.4625
  R2: 0.06839998583117868
Model =  RandomForestRegressor()
  RMSE: 0.5783521850914025
  MAE: 0.41662499999999997
  R2: 0.5260653182671533


In [22]:
logged_model = 'runs:/dc410285d1f949f7b695b1eeecd2ee49/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(test_x))

array([6.22, 5.41, 5.77, 6.25, 5.86, 5.13, 6.09, 6.84, 5.11, 5.28, 6.22,
       5.51, 5.53, 6.72, 5.12, 6.25, 5.61, 5.54, 5.14, 5.04, 5.07, 5.68,
       5.22, 5.09, 6.84, 5.34, 5.9 , 5.21, 5.34, 5.37, 5.42, 5.76, 5.18,
       5.13, 5.06, 5.79, 5.99, 4.79, 5.13, 5.9 , 5.08, 5.97, 5.05, 6.  ,
       6.15, 5.24, 5.79, 6.01, 5.88, 5.07, 6.84, 5.11, 5.05, 5.42, 5.85,
       5.12, 5.74, 5.28, 6.02, 5.31, 5.71, 5.74, 5.67, 5.62, 5.26, 4.98,
       4.79, 5.58, 6.5 , 6.59, 5.2 , 5.1 , 4.91, 6.81, 6.17, 5.6 , 5.12,
       5.85, 5.19, 6.51, 5.8 , 6.02, 5.81, 5.17, 5.83, 5.04, 5.73, 4.97,
       5.89, 5.06, 5.69, 6.91, 5.66, 4.82, 6.03, 5.58, 5.22, 6.54, 5.46,
       5.82, 5.4 , 5.56, 5.25, 5.96, 6.9 , 5.42, 6.19, 5.69, 5.18, 5.99,
       6.98, 5.08, 5.42, 5.95, 5.54, 5.26, 5.49, 6.02, 6.16, 5.2 , 5.16,
       5.44, 4.91, 5.21, 5.31, 5.82, 5.32, 5.25, 6.05, 5.27, 5.47, 5.45,
       6.38, 5.04, 5.69, 5.41, 5.56, 5.82, 5.31, 5.08, 5.95, 5.13, 5.77,
       5.45, 5.22, 6.04, 5.33, 5.52, 6.79, 6.74, 5.